# Projeto 1 - Ciência dos Dados

Nome: Arthur Martins de Souza Barreto

Nome: Giselle Vieira de Melo

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___

Os gênios do crime que dominaram as telas do mundo inteiro em <em><b> La casa de Papel </b></em>  e que conquistaram várias premiações como o Emmy Internacional de melhor série dramática, além de se tornar uma das séries mais populares da IMDb possui uma boa aclamação crítica dado seu enredo sofisticado e dramas interpessoais, se tornando a série em língua não inglesa mais assistida de 2018. 

A partir disso, com o lançamento da primeira parte da quinta (e última) temporada no dia 03 de Setembro de 2021, é de regular tendência pelo mundo inteiro comentários no twitter, tendo até celebridades comentando sobre a série nessa rede. 

<center> <img src="imagens/Money-Heist.jpg" width=500> <center> 

Dessa forma, por meio da categorização dos postagens dos usuários do twitter, esse projeto visa analisar os comentários a respeito da obra, como elogios e críticas diretamente relacionados à série, desconsiderando por exemplo, tweets de marcação ou sobre temas paralelos. 

Para tanto, o <em><b>Classificador Naive Bayes </b></em> foi utilizado!

___
Carregando algumas bibliotecas:

In [1]:
# a biblioteca dos emojis precisa ser baixada, caso não tenha baixado descomente a linha a baixo e faça o dowload
#pip install emoji 

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
# bibliotecas adicionais
import re 
# para eliminar as preposições usei o link a seguir como referencia
# https://stackoverflow.com/questions/29523254/python-remove-stop-words-from-pandas-dataframe
import nltk 
# apra eliminar os emojis usei o seguinte link como referencia
# https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
import emoji

In [4]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\arthu\Desktop\Documentos\Insper\2º_Periodo\Ciencia_dos_dados\twitter


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [5]:
filename = 'La casa de papel.xlsx'

In [6]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevante
0,coisas q serão proibidas quando eu for preside...,1
1,agr que tô terminando de assistir la casa de p...,1
2,paguem minha terapia (la casa de papel vc me p...,0
3,to com pena de terminar la casa de papel,0
4,"olha, quem me segue aqui sabe o tanto que odei...",1


In [7]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevante
0,vou assistir a última temperada de la casa de ...,1
1,só quero chegar em casa e me entupir de brigad...,1
2,"toda vez, a gnt depois que acaba de ver la cas...",0
3,@esteseverino na verdade eles se comunicavam p...,1
4,não tô chorando horrores com o final de la cas...,0


___
## Classificador automático de sentimento

Nessa parte do projeto é necessário categorizar a opinião daqueles comentários no twitter relativos à aprovação da série, (mais precisamente da nova temporada).

Assim, foram encontrados diversos tweets, sendo alguns destes demasiadamente vagos, onde não dava para discernir se era um comentário aprovando ou não da série La casa de papel. Nesse caso, foi considerado relevante apenas os comentários que deixavam claro a critica à série. 

- Funções utilizadas!

In [8]:
# FUNÇÕES AUXILIARES DO SISTEMA para a montagem da base de dados do treinamento
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;\/)|,''""“”@#(*]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed.lower()

# função copiada do seguinte link: https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
def remove_emoji(string):
    return emoji.get_emoji_regexp().sub(u'', string)

# baixando as preposições
nltk.download('stopwords')
# definindo as preposições da lingua portuguesa
stop = nltk.corpus.stopwords.words('portuguese')
def segunda_limpeza(lis_tweet):
    limpo = []
    for pal in lis_tweet:
        # além de verificar se a palavra não é uma preposição vamos pegar só as que tem mais de 3 letras
        # vamos aproveitar que estamos percorrendo cada palavra e remover os emojis nesse processo
        if pal not in stop and len(pal)>3:
            pal = remove_emoji(pal)
            # pode ser que a palavra seja só o emoji, vamos verificar o tamanho dela de novo,
            # pois pode ser uma palavra vazia
            if len(pal) > 3:
                limpo.append(pal)
        else:
            pass
    return limpo

def frase_para_palavras(df):
    '''
    função para percorrer linha a linha uma coluna do dataframe e concatenar as palavras em uma unica lista
    para ser usada no value_counts()
    '''
    data = ''
    
    for tweet in df:
        # o tweet é a frase que a pessoa postou, que se encontra em cada linha da nossa coluna
        data += tweet
        
    # dando o split agora, para separar por virgula
    data = data.split()
    # data é um vetor, podemos chamar a função de tirar preposição aqui
    data = segunda_limpeza(data)
    return data

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


___


## Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

O classificador Naive-Bayes é da família de classificadores estatísticos baseado no Teorema de Bayes, sendo utilizado para prever a ocorrência em um texto, muito útil para esse trabalho já que estamos tratando uma sequência de tweets sobre o tema em questão "La casa de papel". 

Assim, esse classificador, ao ter um conhecimento prévio das condições - aqui dadas por relevante (1) e irrelevante (0) -  utiliza da frequência de palavras para fazer sua classificação, já que ele trata cada palavra de uma frase como um termo independente, como por exemplo: "la casa de papel é muito bom" é equivalente a "é muito bom la casa de papel".

### O teorema de Bayes e as probabilidades independentes:
"Os eventos A e B são independentes quando o fato de ter conhecimento sobre a ocorrência de A não altera a expectativa sobre a probabilidade de ocorrência do evento B."

Isso é muito útil para o nosso modelo já que a probabilidade de que as mesmas frases sejam ditas em postagens distintas no twitter é baixa, mas ao fazer a suposição de que cada palavra em uma frase atua de forma independente é possível se realizar os cálculos de probabilidade.

### Cálculos de probabilidade

- legenda:

    - $P(rel)$: probabilidade do post ser relevante;

    - $P(irrel)$: probabilidade do post ser irrelevante;

    - $P(total)$: $P(rel)$ + $P(irrel)$

    - $P(rel|total)$: 
    $$P(rel|total) = \frac{P(total|rel) P(rel)}{P(total)}$$
    
    - $P(irrel|total)$: 
    $$P(irrel|total) = \frac{P(total|irrel) P(irrel)}{P(total)}$$

#### Suavização de Laplace
Pelo cálculo das probabilidades independentes tem-se que, a partir de uma palavra, descobrir qual é a probabilidade de essa palavra está na base de dados a partir de sua frequência. Agora imagine que a palavra não está na base de dados... o que ocorre é que a probabilidade é zero, e ao realizar a multiplicação dos termos, o cálculo é anulado, não gerando nenhuma informação (o contrário do que se espera).

Assim, a suavização de Laplace surge para prevenir esse caso, dado que é adicionado 1 a cada contagem, para que o denominador nunca fique nulo. Em contrapartida, é também adicionado o número de palavras totais ao divisor, o que leva a, ao realizar a divisão, gerar sempre um número menor que um. 



In [9]:
#convertendo em variáveis categóricas:
train['Treinamento'] = train['Treinamento'].astype('category')
test['Teste'] = test['Teste'].astype('category')

In [10]:
#limpando pontuações do dataframe
train['Clean']=train['Treinamento'].apply(cleanup)
test['Clean']=test['Teste'].apply(cleanup)

In [11]:
test

,Teste,Relevante,Clean
0,vou assistir a última temperada de la casa de ...,1,vou assistir a última temperada de la casa de ...
1,só quero chegar em casa e me entupir de brigad...,1,só quero chegar em casa e me entupir de brigad...
2,"toda vez, a gnt depois que acaba de ver la cas...",0,toda vez a gnt depois que acaba de ver la casa...
3,@esteseverino na verdade eles se comunicavam p...,1,esteseverino na verdade eles se comunicavam pe...
4,não tô chorando horrores com o final de la cas...,0,não tô chorando horrores com o final de la cas...
...,...,...,...
195,acabei la casa de papel,0,acabei la casa de papel
196,precisamos conversar sobre o final dessa tempo...,0,precisamos conversar sobre o final dessa tempo...
197,"eu odeio lá casa de papel, mataram a tóquio va...",1,eu odeio lá casa de papel mataram a tóquio vai...
198,preguiça de ver essa parte 5 de la casa de papel,0,preguiça de ver essa parte 5 de la casa de papel


In [12]:
# post relevante
post_rel = train.loc[(train['Relevante'] == 1)]
# post irrelevante
post_irrel = train.loc[(train['Relevante'] == 0)]

In [13]:
# agora precisamos quebrar o texto em palavras, usando a função aux que converter frase em palavras,
# ao mesmo tempo, vamos retirar as preposições do código, paralelamente a conversão, melhorando a base de dados, temos:
data_rel   = frase_para_palavras(post_rel['Clean'])
data_irrel = frase_para_palavras(post_irrel['Clean'])

# criando a serie dos posts relevantes e irrelevantes
serie_rel = pd.Series(data_rel)
serie_irrel = pd.Series(data_irrel)
# Formando uma base de daddos com todas as palavras nele:
data_total = data_rel + data_irrel

#criando a serie com a base total de dados
serie_total = pd.Series(data_total)

print(len(serie_rel))
print(len(serie_irrel))
print(len(serie_total))
serie_irrel.tail(20)

1214
906
2120


886            moto
887        guardada
888        deitadão
889           vendo
890            casa
891           papel
892       trabalhar
893          recebi
894         spoiler
895            casa
896           papel
897           quero
898         cometer
899    homicídiovou
900      aproveitar
901           novos
902       episódios
903            casa
904           papel
905      abafadinha
dtype: object

#### Frequências absolutas

Palavras em um texto são variáveis **qualitativas nominais**, portanto usaremos `value_counts()` para obter a tabela de frequências relativas e absolutas:

In [14]:
# construindo a tabela de relevante e irrelevante
tabela_rel   = serie_rel.value_counts()
tabela_irrel = serie_irrel.value_counts()
tabela_total = serie_total.value_counts()
lista_serie_total = serie_total.tolist()
elementos_nao_repetidos = set(lista_serie_total)
elementos_nao_repetidos = pd.DataFrame(elementos_nao_repetidos).value_counts()
elementos_nao_repetidos

100odeio         1
papelia          1
papelagora       1
papelarturo      1
papelassisti     1
                ..
facebook         1
falando          1
falar            1
falecermaaano    1
único            1
Length: 952, dtype: int64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [15]:
# postagens relevantes
post_rel.head()

,Treinamento,Relevante,Clean
0,coisas q serão proibidas quando eu for preside...,1,coisas q serão proibidas quando eu for preside...
1,agr que tô terminando de assistir la casa de p...,1,agr que tô terminando de assistir la casa de p...
4,"olha, quem me segue aqui sabe o tanto que odei...",1,olha quem me segue aqui sabe o tanto que odeio...
6,que final fdp do lá casa de papel,1,que final fdp do lá casa de papel
8,"final veio de lá casa de papel, meu pai amado ...",1,final veio de lá casa de papel meu pai amado o...


In [16]:
# postagens irrelevantes
post_irrel.head()

,Treinamento,Relevante,Clean
2,paguem minha terapia (la casa de papel vc me p...,0,paguem minha terapia la casa de papel vc me paga
3,to com pena de terminar la casa de papel,0,to com pena de terminar la casa de papel
5,simplesmente devastada com esse final da 1° pa...,0,simplesmente devastada com esse final da 1° pa...
7,ai la casa de papel me mata pqp,0,ai la casa de papel me mata pqp
10,criadores de lá casa de papel deixaram mto na ...,0,criadores de lá casa de papel deixaram mto na ...


In [17]:
p_rel = len(serie_rel)/len(serie_total)
print('A probabilidade de relevantes é: {0:.3f}'.format(p_rel))

A probabilidade de relevantes é: 0.573


In [18]:
p_irrel = len(serie_irrel)/len(serie_total)
print('A probabilidade de irrelevate é: {0:.3f}'.format(p_irrel))

A probabilidade de irrelevate é: 0.427


### Funções auxiliares para a suavização de la place

In [19]:
def quantas_vezes_aparece_palavra(palavra,tabela):
    # a função retorna o valor de vezes que a palavra aparece no seu conjunto
    # por exemplo, quantas vezes aparece na tabela relevante?
    # fazemos: num = tabela_rel['palavra']
    # e retornamos esse valor
    # por outro lado, pode não ter a palavra, por isso a suaviação de la place, quando isso ocorrer,
    # vamos retornar 0
    try:
        # caso em que a palavra tem na tabela de frequencias
        quant = tabela[palavra]
        return quant
    except:
        # aqui a palavra não está na tabela de frequencias,retornamos 0
        return 0
    
def laplace_smoothing(palavras_da_classe,quantidade):
    # vamos aplicar a formula da suavização
    num = quantidade + 1
    den = len(palavras_da_classe) + len(elementos_nao_repetidos)
    #den = 2193 + len(elementos_nao_repetidos)
    laplace = num/den
    return laplace

def p_tweet(tweet,relevante):
    # se relevante for true, calcula para relevante
    # se for false, para irrelevante
    p = 1
    # quebrando o tweet a cada espaço 
    tweet_em_palavras = tweet.split()
    
    for palavra in tweet_em_palavras:
        # vamos calcular o valor da frequencia absoluta da palavra
        # para isto, usaremos a funcao quantas_vezes_aparece_palavra
        # verificando qual classe queremos calcular p
        
        if relevante:
            quant = quantas_vezes_aparece_palavra(palavra,tabela_rel)
            p_palavra = laplace_smoothing(serie_rel,quant)
            
        else:
            quant = quantas_vezes_aparece_palavra(palavra,tabela_irrel)
            p_palavra = laplace_smoothing(serie_irrel,quant)
            
        p *= p_palavra
    return p

def rel_ou_irrel(tweet):
    # vamos decidir se o tweet é relevante ou não
    # para tal vamos aplicar o teorema de bayes
    P_tweet_rel   = p_tweet(tweet,True)
    P_tweet_irrel = p_tweet(tweet,False)
    rel_bayes     = P_tweet_rel*p_rel
    irrel_bayes   = P_tweet_irrel*p_irrel
    #print(rel_bayes,irrel_bayes)
    # vamos usar o mesmo criterio adotado na tabela, 1 para relevante e 0 para irrelevante
    if rel_bayes > irrel_bayes:
        return 1
    else:
        return 0

In [20]:
# agora vamos criar uma coluna nova, com a classificação dos tweets
train['Classificação'] = train['Clean'].apply(rel_ou_irrel)
train

,Treinamento,Relevante,Clean,Classificação
0,coisas q serão proibidas quando eu for preside...,1,coisas q serão proibidas quando eu for preside...,1
1,agr que tô terminando de assistir la casa de p...,1,agr que tô terminando de assistir la casa de p...,1
2,paguem minha terapia (la casa de papel vc me p...,0,paguem minha terapia la casa de papel vc me paga,0
3,to com pena de terminar la casa de papel,0,to com pena de terminar la casa de papel,0
4,"olha, quem me segue aqui sabe o tanto que odei...",1,olha quem me segue aqui sabe o tanto que odeio...,1
...,...,...,...,...
295,mó sacanagem esse final de la casa de papel,1,mó sacanagem esse final de la casa de papel,1
296,recebi spoiler de la casa de papel e quero com...,0,recebi spoiler de la casa de papel e quero com...,0
297,terminei la casa de papel e me tornei o homem ...,1,terminei la casa de papel e me tornei o homem ...,1
298,vou aproveitar pra ver os novos episódios de l...,0,vou aproveitar pra ver os novos episódios de l...,0


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**